In [30]:
import numpy as np
import pandas as pd
import os, sys
sys.path.insert(0, "../modules")
import l96
import convert as cv
import experiment as ex
import utility as ut
import evaluate as ev

ds = "L96"
root = f"../data/{ds}/autonomous-all-in"
df_train = pd.read_csv(f"{root}/train.csv")
train = np.load(f"{root}/train.npy")
D_r = 256

train_kwargs = {'save_folder': f"{root}/autonomous-all-in", "epochs": 50000, "I": 4}
data_gen_kwargs = {"dt": 1e-2, "train_size": int(2e5), "train_seed": np.random.randint(1e6), "test_num": 1}
feature_cols = [f"Space_{i}" for i in range(2*train_kwargs["I"]+1)]
bnf_kwargs   = {"width": D_r,
                "depth": 2,
                "freq": None,
                "seasonality_periods": None,
                "num_seasonal_harmonics": None,
                "feature_cols": [f"Space_{i}" for i in range(9)],
                "target_col": "Output",
                "observation_model": 'NORMAL',
                "timetype": 'float',
                "standardize":  None,
                "interactions": [(i, j) for i in range(len(feature_cols)) for j in range(len(feature_cols)) if i < j]}
model = ut.load(os.path.abspath(f"{root}/bnf_model"), df_train, **bnf_kwargs)


Time taken by load is 2.3211 seconds


In [39]:
data = np.load(f"{root}/data.npy")
n_steps = 100
N = int(data.shape[1]/2)
x = data[:, N]
y = data[:, N:N+n_steps]
std = train.std(axis=1)
y_ = ev.multistep_forecast(model, 0, x, 4, n_steps, 0.01)
# print(np.sqrt((((y-y_)/std)**2).sum()))


Time taken by multistep_forecast is 37.4621 seconds


In [40]:
x - y_[:, 0]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0.])

In [53]:
np.argmax((((y-y_) / std[:, None])**2).sum(axis=0) > 0.5**2) * 2.27 * 0.01

0.1135

In [63]:
np.argmax((((y-y_) / std[:, None])**2).mean(axis=0) > 0.25) * 0.0227


1.7933000000000001

In [64]:
y__ = np.load(f"{root}/rmse_trajectory.npy")

In [65]:
y_.shape

(40, 100)

In [74]:
np.sqrt(((y - y__)**2).mean(axis=0).mean())

0.2668793577483595

In [76]:
np.abs(y- y__).sum(axis=0).mean()

7.91278591412433